In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time

options = webdriver.EdgeOptions()
options.add_argument('--inprivate')  

driver = webdriver.Edge(executable_path='C:\\Users\\hanno\\Downloads\\edgedriver_win644\\msedgedriver', options=options)

driver.get('https://www.avito.ma/fr/maroc/appartements')


page_limit = 100  

data = []

for page_number in range(1, page_limit + 1):

    driver.get(f'https://www.avito.ma/fr/maroc/appartements?o={page_number}')
    
    print('-------------------------', page_number, '------------------------')

    try:

        src = driver.page_source
        soup = BeautifulSoup(src, 'html.parser')


        ds = soup.find_all('a', class_='sc-1jge648-0')

        for i in range(len(ds)):
            try:
                noms = []
                prix = []
                villes = []
                dates = []
                location = []
                title = []
                span = []
                span_1 = []
                span_2 = []


                noms.append(ds[i].find('p', {'class': 'sc-1x0vz2r-0 iHApav'}).get_text().replace('\n', ''))
                prix.append(ds[i].find('div', {'class': 'sc-b57yxx-0 ckugTG'}).find('div', {'class': 'sc-b57yxx-4 dRjnHr'}).get_text().strip().replace('\n', ''))
                villes.append(ds[i].find('div', {'class': 'sc-b57yxx-10 fHMeoC'}).find('p').get_text())
                dates.append(ds[i].find('div', {'class': 'sc-1wnmz4-2 iDkUpG'}).find('p').get_text())


                url2 = ds[i].get('href')
                driver.get(url2)

                src2 = driver.page_source
                soup2 = BeautifulSoup(src2, 'html.parser')

                location_element = soup2.find('div', class_='sc-1g3sn3w-7 bNWHpB')
                if location_element:
                    location_text = location_element.find_all('span')[0].get_text()
                    location.append(location_text)
                else:
                    location.append('None')

                ds2 = soup2.find('div', class_='sc-1g3sn3w-3 fHaZgB')

                
                for j in range(len(ds2.find_all('div', {'class': 'sc-6p5md9-1'}))):
                    title.append(ds2.find_all('div', {'class': 'sc-6p5md9-1'})[j].find('title').get_text()[:-5])
                    span.append(ds2.find_all('div', {'class': 'sc-6p5md9-1'})[j].find('span').get_text())

                informations = ds2.find('div', class_='sc-qmn92k-0').find_all('li')
                for k in range(len(informations)):
                    span_1.append(informations[k].find_all('span')[0].get_text())
                    span_2.append(informations[k].find_all('span')[1].get_text())

                
                data_dict = {
                    'Nom': noms[0],
                    'Prix': prix[0],
                    'Ville': villes[0],
                    'Date': dates[0],
                    'Location': location[0],
                }

                data_dict.update({title[i]: span[i] for i in range(len(title))})
                data_dict.update({span_1[i]: span_2[i] for i in range(len(span_1))})

                data.append(data_dict)

            except Exception as e:
                print(f"Error extracting data for offer {i} on page {page_number}: {e}")
                continue  

        time.sleep(2)  

    except Exception as e:
        print(f"Error scraping page {page_number}: {e}")
        continue  



C:\Users\hanno\AppData\Local\Temp\ipykernel_17964\286899826.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path='C:\\Users\\hanno\\Downloads\\edgedriver_win644\\msedgedriver', options=options)


------------------------- 1 ------------------------
------------------------- 2 ------------------------
------------------------- 3 ------------------------
------------------------- 4 ------------------------
------------------------- 5 ------------------------
------------------------- 6 ------------------------
------------------------- 7 ------------------------
------------------------- 8 ------------------------
------------------------- 9 ------------------------
------------------------- 10 ------------------------
------------------------- 11 ------------------------
------------------------- 12 ------------------------
------------------------- 13 ------------------------
------------------------- 14 ------------------------
------------------------- 15 ------------------------
------------------------- 16 ------------------------
------------------------- 17 ------------------------
------------------------- 18 ------------------------
------------------------- 19 --------

In [2]:
data

[{'Nom': 'Joli appartement a vendre à Emile Zola belvédère',
  'Prix': '1\u202f650\u202f000 DH',
  'Ville': 'Appartements dans Casablanca, Belvédère',
  'Date': 'il y a 6 minutes',
  'Location': 'Casablanca',
  'Chambres': '4',
  'SalleDeBain': '2',
  'Type': 'Appartements, à vendre',
  'Secteur': 'Belvédère',
  'Salons': '2',
  'Surface habitable': '210',
  'Étage': '7+'},
 {'Nom': 'À Cfc Appartement avec 4 chambres meublé',
  'Prix': '15\u202f000 DH',
  'Ville': 'Appartements dans Casablanca, Casablanca Finance City',
  'Date': 'il y a 6 minutes',
  'Location': 'Casablanca',
  'Chambres': '3',
  'SalleDeBain': '2',
  'Type': 'Appartements, à louer',
  'Secteur': 'Casablanca Finance City',
  'Salons': '1',
  'Surface habitable': '110',
  'Étage': '1'},
 {'Nom': 'CMN-MA-1757 - Appartement à vendre à Palmier',
  'Prix': '1\u202f500\u202f000 DH',
  'Ville': 'Appartements dans Casablanca, Palmier',
  'Date': 'il y a 7 minutes',
  'Location': 'Casablanca',
  'Chambres': '2',
  'SalleDeBain

In [3]:
df = pd.DataFrame(data)
df.to_csv('avito_apartments.csv', index=False)

In [4]:
df

,Nom,Prix,Ville,Date,Location,Chambres,SalleDeBain,Type,Secteur,Salons,Surface habitable,Étage,SurfaceTotale,Capacite,Immo,Horaire d'arrivée,Horaire de départ,Frais de syndic / mois,Âge du bien,Caution
0,Joli appartement a vendre à Emile Zola belvédère,1 650 000 DH,"Appartements dans Casablanca, Belvédère",il y a 6 minutes,Casablanca,4,2,"Appartements, à vendre",Belvédère,2,210,7+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,À Cfc Appartement avec 4 chambres meublé,15 000 DH,"Appartements dans Casablanca, Casablanca Finan...",il y a 6 minutes,Casablanca,3,2,"Appartements, à louer",Casablanca Finance City,1,110,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CMN-MA-1757 - Appartement à vendre à Palmier,1 500 000 DH,"Appartements dans Casablanca, Palmier",il y a 7 minutes,Casablanca,2,1,"Appartements, à vendre",Palmier,NaN,98,4,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,appart calme bien meubler,DownwardArrow IconPrix à la baisse250 DH,"Locations de vacances dans Marrakech, Guéliz",il y a 6 minutes,Marrakech,1,NaN,"Locations de vacances, à louer",Guéliz,NaN,NaN,NaN,NaN,4,Appartements,14:00,12:00,NaN,NaN,NaN
4,شقة مجهزة بالكامل للكراء اليومي,350 DH,"Locations de vacances dans Marrakech, Allal El...",il y a 6 minutes,Marrakech,2,NaN,"Locations de vacances, à louer",Allal El Fassi,NaN,NaN,NaN,NaN,5,Appartements,12:00,12:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,Appartement dans résidence avec piscine,4 500 DH,"Appartements dans Bouznika, Autre secteur",il y a 12 heures,Bouznika,2,2,"Appartements, à louer",Autre secteur,1,65,2,68,NaN,NaN,NaN,NaN,NaN,NaN,1 mois
3696,"appartement à agadir , Hay Salam",500 DH,"Locations de vacances dans Agadir, Hay Salam",il y a 12 heures,Agadir,2,NaN,"Locations de vacances, à louer",Hay Salam,NaN,NaN,NaN,NaN,5+,Appartements,NaN,NaN,NaN,NaN,NaN
3697,appartement tranquille,250 DH,"Locations de vacances dans Bouznika, Quartier ...",il y a 12 heures,Bouznika,2,NaN,"Locations de vacances, à louer",Quartier Industriel,NaN,NaN,NaN,NaN,5+,Appartements,11:00,13:00,NaN,NaN,NaN
3698,Appartement de rêve à vendre à Ain Atiq.,Prix non spécifié,"Appartements dans Temara, Ain Atig",il y a 11 heures,Temara,3,2,"Appartements, à vendre",Ain Atig,1,105,1,NaN,NaN,NaN,NaN,NaN,NaN,Neuf,NaN
